In [ ]:
filepath = '/home/adityasidharta/git/datascience2018/datasets'
gauge_path = filepath+'/gauge.csv'
radar_path = filepath+'/radar'

In [ ]:
from tqdm import tqdm_notebook as tqdm
from scipy.stats import pearsonr

In [ ]:
import pandas as pd
import numpy as np
import oscd

In [ ]:
df_gauge = pd.read_csv(gauge_path)

In [ ]:
df_result = pd.DataFrame()
train_week = []
for filename in tqdm(os.listdir(radar_path)):
    week = filename.split('.')[0][-2:]
    train_week.append(str(week))
    full_radar_path = os.path.join(radar_path, filename)
    df_radar = pd.read_csv(full_radar_path)
    radar_obs = df_radar.values[:,2:]
    radar_arange = np.arange(0, 34)
    radar_mean = np.matmul(radar_obs, radar_arange) / 2016.
    if 'latlong' not in df_result.columns:
        df_result['latlong'] = df_radar['lat'] * 1000 + df_radar['long']
    assert np.allclose(df_result['latlong'].values, df_radar['lat'] * 1000 + df_radar['long'])
    df_result['mean_week_'+str(week)] = radar_mean

In [ ]:
train_week

In [ ]:
df_result = df_result[sorted(df_result.columns)]

In [ ]:
df_result.head()

In [ ]:
df_gauge = pd.read_csv(gauge_path)

In [ ]:
len(np.where(pd.isnull(df_gauge))[0])

In [ ]:
df_gauge['18']

In [ ]:
df_gauge['Year'] = df_gauge['Year'].astype(str)
df_gauge['Week'] = df_gauge['Week'].astype(str)
df_gauge = df_gauge.apply(lambda x: x.fillna(x[2:].mean()),axis=1)

In [ ]:
train_week = sorted(train_week)

In [ ]:
train_bool = df_gauge['Week'].apply(lambda x : x in train_week)
train_df_gauge = df_gauge[train_bool]

In [ ]:
train_df_gauge_array = train_df_gauge.values[:, 2:]

In [ ]:
train_df_gauge.head()

In [ ]:
train_df_gauge_array.shape


In [ ]:
df_result['latlong'].values

In [ ]:
print np.where(pd.isnull(train_df_gauge_array))
print np.where(pd.isnull(df_result))

In [ ]:
df_corr_result = pd.DataFrame()
df_corr_result['latlong'] = df_result['latlong'].values
for station in tqdm(range(train_df_gauge_array.shape[1])):
    station_values = train_df_gauge_array[:, station]
    total_latlong = []
    for idx, latlong in tqdm(df_result.iterrows()):
        latlong_values = latlong.values[1:]
        correlation = np.corrcoef(station_values.tolist(), latlong_values.tolist())[0][1]
        total_latlong.append(correlation)
    df_corr_result['corr_station_'+str(station)] = total_latlong

In [ ]:
df_corr_result.to_csv('adit_csv')

In [ ]:
df_corr_result = pd.read_csv('adit.csv')
del df_corr_result['Unnamed: 0']

In [ ]:
df_corr_result.sort('corr_station_0',ascending = False)

In [ ]:
len(np.where(pd.isnull(df_corr_result))[0])

In [ ]:
latlong = df_corr_result.values[:,0]

In [ ]:
df_corr_result_array = df_corr_result.values[:,1:]
df_corr_result_array.shape

In [ ]:
def to_lat(latlong):
    return int(latlong // 1000)

def to_long(latlong):
    return int(latlong % 1000)

In [ ]:
dict_result = {}
for station in range(df_corr_result_array.shape[1]):
    values = df_corr_result_array[:, station]
    index = np.nanargmax(values)
    dict_result[str(station)] = {
    'lat' : to_lat(latlong[index]),
    'long': to_long(latlong[index])
    }

In [ ]:
import json
with open('station_location.json', 'w') as outfile:
    json.dump(dict_result, outfile)

In [ ]:
dict_result